In [41]:
%%bash

mkdir -p tmp/train/0 tmp/train/1 tmp/train/2 tmp/train/3 tmp/train/4
mkdir -p tmp/train/5 tmp/train/6 tmp/train/7 tmp/train/8 tmp/train/9
mkdir -p tmp/validation/0 tmp/validation/1 tmp/validation/2 tmp/validation/3 tmp/validation/4
mkdir -p tmp/validation/5 tmp/validation/6 tmp/validation/7 tmp/validation/8 tmp/validation/9

In [42]:
%%bash

mkdir -p tmp/train_lst
mkdir -p tmp/validation_lst
mkdir -p tmp/test

In [43]:
%%bash

ls -1F tmp/train

0/
1/
2/
3/
4/
5/
6/
7/
8/
9/


In [44]:
%%bash

ls -1F tmp/validation

0/
1/
2/
3/
4/
5/
6/
7/
8/
9/


In [45]:
import mxnet as mx
mx.random.seed(21)

In [46]:
def transform_fxn(data, label):
    data = data.astype('float32')
    data = data / 255
    return data, label

ds = mx.gluon.data.vision.datasets.MNIST(
    train=True, 
    transform=transform_fxn
)
training_and_validation_dataset = ds

ds = mx.gluon.data.vision.datasets.MNIST(
    train=False, 
    transform=transform_fxn
)
test_dataset = ds

In [47]:
print(len(training_and_validation_dataset))
print(len(test_dataset))

60000
10000


In [48]:
def get_training_row_indexes(row_count, 
                             percent=0.5, 
                             ratio=0.8):
    training_index_start = 0
    end = int(row_count * ratio * percent)
    training_index_end = end
    
    print("Range Index Start:", 
          training_index_start)
    print("Range Index End:", 
          training_index_end)
    
    output = list(range(training_index_start, 
                        training_index_end))
    
    print("Output Length:", len(output))
    print("Last Index:", output[-1])
    
    return output

In [49]:
def get_validation_row_indexes(row_count, 
                               percent=0.5, 
                               ratio=0.8):
    start = int(row_count * ratio)
    validation_index_start = start
    
    count = int((1 - ratio) * row_count * percent) + 1
    element_count = count
    validation_index_end = validation_index_start + element_count
    
    print("Range Index Start:", 
          validation_index_start)
    print("Element Count:", 
          element_count)
    print("Range Index End:", 
          validation_index_end)
    
    output = list(range(validation_index_start, 
                        validation_index_end))
    
    print("Output Length:", len(output))
    print("Last Index:", output[-1])
    
    return output

In [50]:
def get_test_row_indexes(row_count, 
                         percent=0.5):
    test_index_start = 0
    test_index_end = int(row_count * percent)
    
    print("Range Index Start:", 
          test_index_start)
    print("Range Index End:", 
          test_index_end)
    
    output = list(range(test_index_start, 
                        test_index_end))
    
    print("Output Length:", len(output))
    print("Last Index:", output[-1])
    
    return output

In [51]:
get_training_row_indexes(row_count=60000, 
                         percent=0.5)

Range Index Start: 0
Range Index End: 24000
Output Length: 24000
Last Index: 23999


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [52]:
get_validation_row_indexes(row_count=60000, 
                           percent=0.5)

Range Index Start: 48000
Element Count: 6000
Range Index End: 54000
Output Length: 6000
Last Index: 53999


[48000,
 48001,
 48002,
 48003,
 48004,
 48005,
 48006,
 48007,
 48008,
 48009,
 48010,
 48011,
 48012,
 48013,
 48014,
 48015,
 48016,
 48017,
 48018,
 48019,
 48020,
 48021,
 48022,
 48023,
 48024,
 48025,
 48026,
 48027,
 48028,
 48029,
 48030,
 48031,
 48032,
 48033,
 48034,
 48035,
 48036,
 48037,
 48038,
 48039,
 48040,
 48041,
 48042,
 48043,
 48044,
 48045,
 48046,
 48047,
 48048,
 48049,
 48050,
 48051,
 48052,
 48053,
 48054,
 48055,
 48056,
 48057,
 48058,
 48059,
 48060,
 48061,
 48062,
 48063,
 48064,
 48065,
 48066,
 48067,
 48068,
 48069,
 48070,
 48071,
 48072,
 48073,
 48074,
 48075,
 48076,
 48077,
 48078,
 48079,
 48080,
 48081,
 48082,
 48083,
 48084,
 48085,
 48086,
 48087,
 48088,
 48089,
 48090,
 48091,
 48092,
 48093,
 48094,
 48095,
 48096,
 48097,
 48098,
 48099,
 48100,
 48101,
 48102,
 48103,
 48104,
 48105,
 48106,
 48107,
 48108,
 48109,
 48110,
 48111,
 48112,
 48113,
 48114,
 48115,
 48116,
 48117,
 48118,
 48119,
 48120,
 48121,
 48122,
 48123,
 48124,


In [53]:
get_test_row_indexes(row_count=10000, 
                     percent=0.1)

Range Index Start: 0
Range Index End: 1000
Output Length: 1000
Last Index: 999


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [54]:
import string 
import random

def generate_random_string():
    return ''.join(
        random.sample(
        string.ascii_uppercase,12)
    )


In [55]:
generate_random_string()

'FAOJWDNPKLIY'

In [56]:
import matplotlib
import matplotlib.pyplot

def save_image(image_data, filename):
    matplotlib.pyplot.imsave(
        f"tmp/{filename}", 
        image_data[:,:,0].asnumpy())

In [57]:
def generate_image_files_and_lst_dict(
    dataset, 
    indexes, 
    tag
):
    list_of_lst_dicts = []
    
    for index in indexes:
        image_label_pair = dataset[index]
        image_data = image_label_pair[0]
        label = image_label_pair[1]
        random_string = generate_random_string()

        if tag == "test":
            rp = f"{random_string}.png"
            relative_path = rp
            filename = f"{tag}/{relative_path}"
        else:
            rp = f"{label}/{random_string}.png"
            relative_path = rp
            filename = f"{tag}/{relative_path}"

        save_image(
            image_data, 
            filename=filename
        )
        
        lst_dict = {
            'relative_path': relative_path, 
            'class': label
        }
        list_of_lst_dicts.append(lst_dict)

    return list_of_lst_dicts

In [58]:
train_dataset_length = len(
    training_and_validation_dataset
)
train_indexes = get_training_row_indexes(
    row_count=train_dataset_length, 
    percent=0.01)

t = generate_image_files_and_lst_dict(
    dataset=training_and_validation_dataset,
    indexes=train_indexes,
    tag = "train"
)
train_lst_dict = t

Range Index Start: 0
Range Index End: 480
Output Length: 480
Last Index: 479


In [59]:
train_lst_dict

[{'relative_path': '5/YVPWIDFZOKHB.png', 'class': 5},
 {'relative_path': '0/MSKWHRLVNIZO.png', 'class': 0},
 {'relative_path': '4/SJEUAXWNCYVL.png', 'class': 4},
 {'relative_path': '1/XCFGURMWESPO.png', 'class': 1},
 {'relative_path': '9/XSYAGICOHJME.png', 'class': 9},
 {'relative_path': '2/SCALZKIVHOPY.png', 'class': 2},
 {'relative_path': '1/FEWDZXVRYCUN.png', 'class': 1},
 {'relative_path': '3/RSFMGVUIQEYO.png', 'class': 3},
 {'relative_path': '1/BWUIXJEHYTMR.png', 'class': 1},
 {'relative_path': '4/DPNZYULEKBXS.png', 'class': 4},
 {'relative_path': '3/GCBAORQFDLWJ.png', 'class': 3},
 {'relative_path': '5/TQYNZIAESDFW.png', 'class': 5},
 {'relative_path': '3/LEHZKTWGDARS.png', 'class': 3},
 {'relative_path': '6/IUNPLSAFYKZV.png', 'class': 6},
 {'relative_path': '1/SFKRHVAZYPUI.png', 'class': 1},
 {'relative_path': '7/PNLACVGSOYDW.png', 'class': 7},
 {'relative_path': '2/OYDUGZWVJIHS.png', 'class': 2},
 {'relative_path': '8/EZHOCRQPKSTI.png', 'class': 8},
 {'relative_path': '6/DOTUPB

In [60]:
train_dataset_length = len(
    training_and_validation_dataset
)
validation_indexes = get_validation_row_indexes(
     row_count=train_dataset_length, 
     percent=0.01)

v = generate_image_files_and_lst_dict(
    dataset=training_and_validation_dataset,
    indexes=validation_indexes,
    tag = "validation"
)
validation_lst_dict = v

Range Index Start: 48000
Element Count: 120
Range Index End: 48120
Output Length: 120
Last Index: 48119


In [61]:
validation_lst_dict

[{'relative_path': '4/UCMTKVGOAHRI.png', 'class': 4},
 {'relative_path': '7/MZGVKIFQJADB.png', 'class': 7},
 {'relative_path': '8/ABLJREPQDCUN.png', 'class': 8},
 {'relative_path': '8/VHUTAYOGNMIP.png', 'class': 8},
 {'relative_path': '5/OUIVPCBKMREN.png', 'class': 5},
 {'relative_path': '6/GKJSTNIHRZYV.png', 'class': 6},
 {'relative_path': '6/CSVZRIEPWOMN.png', 'class': 6},
 {'relative_path': '8/YXAEOUHKZLQM.png', 'class': 8},
 {'relative_path': '2/YZXMJEBCUQPA.png', 'class': 2},
 {'relative_path': '2/KBVIAYNGCMXW.png', 'class': 2},
 {'relative_path': '7/VNFGELBQSYKA.png', 'class': 7},
 {'relative_path': '8/MAGLOJRVBIEC.png', 'class': 8},
 {'relative_path': '0/YRWPLCBZTDIG.png', 'class': 0},
 {'relative_path': '4/QPSOTBNJEKWY.png', 'class': 4},
 {'relative_path': '3/HQDAIZCTXNVE.png', 'class': 3},
 {'relative_path': '4/KNARPFDQLYOX.png', 'class': 4},
 {'relative_path': '5/KCHGMELYIXVW.png', 'class': 5},
 {'relative_path': '2/HLMJUNCTYOFZ.png', 'class': 2},
 {'relative_path': '4/JOTHPZ

In [62]:
test_dataset_length = len(test_dataset)
test_indexes = get_test_row_indexes(
    row_count=test_dataset_length, 
    percent=0.01)

test_lst_dict = generate_image_files_and_lst_dict(
    dataset=test_dataset,
    indexes=test_indexes,
    tag = "test"
)

Range Index Start: 0
Range Index End: 100
Output Length: 100
Last Index: 99


In [63]:
test_lst_dict

[{'relative_path': 'LHOJUCZNYXWF.png', 'class': 7},
 {'relative_path': 'DAVLUBHMORCJ.png', 'class': 2},
 {'relative_path': 'PWXZHDTNCBYS.png', 'class': 1},
 {'relative_path': 'EQTCVYXKGHDP.png', 'class': 0},
 {'relative_path': 'ONGYTSWPDJHK.png', 'class': 4},
 {'relative_path': 'FBUKLQCSTIMJ.png', 'class': 1},
 {'relative_path': 'BTCDFQLEVXIH.png', 'class': 4},
 {'relative_path': 'LRSIEUFPVTMZ.png', 'class': 9},
 {'relative_path': 'MPZNCIASFWKL.png', 'class': 5},
 {'relative_path': 'PIMOQNKHGEXR.png', 'class': 9},
 {'relative_path': 'GOVPSMTQKECF.png', 'class': 0},
 {'relative_path': 'STYGDPHAMXFV.png', 'class': 6},
 {'relative_path': 'ARHPXSVITOGY.png', 'class': 9},
 {'relative_path': 'UFNHIWGALJYX.png', 'class': 0},
 {'relative_path': 'BXYQTDHLAJRI.png', 'class': 1},
 {'relative_path': 'FIBROJMVWDCH.png', 'class': 5},
 {'relative_path': 'LKYTDGPBQSHZ.png', 'class': 9},
 {'relative_path': 'VQWAKPOBDYIL.png', 'class': 7},
 {'relative_path': 'SAZIDVUHQLWY.png', 'class': 3},
 {'relative_

In [64]:
def save_lsts_to_file(values, filename):
    with open(filename, 'w') as output:
        for index, row in enumerate(
            values, 
            start=1
        ):
            relative_path = row['relative_path']
            cls = row['class']
            tmp = f"{index}\t{cls}\t{relative_path}\n"
            output.write(tmp)

In [65]:
save_lsts_to_file(
    train_lst_dict, 
    filename="tmp/train_lst/train.lst"
)
save_lsts_to_file(
    validation_lst_dict, 
    filename="tmp/validation_lst/validation.lst"
)

In [66]:
%%bash

head tmp/train_lst/train.lst

1	5	5/YVPWIDFZOKHB.png
2	0	0/MSKWHRLVNIZO.png
3	4	4/SJEUAXWNCYVL.png
4	1	1/XCFGURMWESPO.png
5	9	9/XSYAGICOHJME.png
6	2	2/SCALZKIVHOPY.png
7	1	1/FEWDZXVRYCUN.png
8	3	3/RSFMGVUIQEYO.png
9	1	1/BWUIXJEHYTMR.png
10	4	4/DPNZYULEKBXS.png


In [67]:
s3_bucket = "sagemaker-cookbook-bucket"
prefix = "image-experiments"
!aws s3 cp tmp/.  s3://{s3_bucket}/{prefix}/ --recursive

upload: tmp/test/ADTPKJMVLWYX.png to s3://sagemaker-cookbook-bucket/image-experiments/test/ADTPKJMVLWYX.png
upload: tmp/test/ARHPXSVITOGY.png to s3://sagemaker-cookbook-bucket/image-experiments/test/ARHPXSVITOGY.png
upload: tmp/test/AGQSRHNYPJTL.png to s3://sagemaker-cookbook-bucket/image-experiments/test/AGQSRHNYPJTL.png
upload: tmp/test/AIRTVDOJGFYZ.png to s3://sagemaker-cookbook-bucket/image-experiments/test/AIRTVDOJGFYZ.png
upload: tmp/test/AYNLXMTUVQSP.png to s3://sagemaker-cookbook-bucket/image-experiments/test/AYNLXMTUVQSP.png
upload: tmp/test/BWKERDJUXLZM.png to s3://sagemaker-cookbook-bucket/image-experiments/test/BWKERDJUXLZM.png
upload: tmp/test/BARWDQHLUFTP.png to s3://sagemaker-cookbook-bucket/image-experiments/test/BARWDQHLUFTP.png
upload: tmp/test/BTCDFQLEVXIH.png to s3://sagemaker-cookbook-bucket/image-experiments/test/BTCDFQLEVXIH.png
upload: tmp/test/BNEHDSICOXLR.png to s3://sagemaker-cookbook-bucket/image-experiments/test/BNEHDSICOXLR.png
upload: tmp/test/BSMLCZXVAEN

In [68]:
%store s3_bucket
%store prefix

Stored 's3_bucket' (str)
Stored 'prefix' (str)
